# 1. Configuration

### A. Importing dependencies

In [17]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import os
from IPython.core.interactiveshell import InteractiveShell

# Enable the display of multiple dataframes in outputs when running code
InteractiveShell.ast_node_interactivity = 'all'

# Display all columns in dataframe outputs
pd.options.display.max_columns = None

### B. Connecting to database

In [18]:
# Connection variables
SERVER = os.getenv('SERVER')
DATABASE = os.getenv('DATABASE')
DRIVER = 'ODBC Driver 17 for SQL Server'

DATABASE_CONNECTION = f'mssql://@{SERVER}/{DATABASE}?driver={DRIVER}'

# Establish connection to the database
engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

### C. Testing database connection

In [19]:
# Commented out when testing is finished

# Testing table creation
#test_table = pd.DataFrame(data={'col1': [1, 2], 'col2': [3, 4]}) 
#test_table.to_sql('Test', con=connection, if_exists='replace', index=False)

# Testing table read
#query = 'select * from Test'
#pd.read_sql(sql=query, con=connection)

### D. Loading file

In [20]:
main_dataframe = pd.read_csv('D:\Bureau\data_project\listings.csv')

# 2. Data cleaning

### A. Dropping unusued columns

In [21]:
columns_to_drop = [
    'listing_url',
    'scrape_id',
    'last_scraped',
    'source',
    'description',
    'neighbourhood',
    'neighborhood_overview',
    'neighbourhood_group_cleansed',
    'picture_url',
    'host_url',
    'host_location',
    'host_about',
    'host_response_time',
    'host_response_rate',
    'host_acceptance_rate',
    'host_is_superhost',
    'host_thumbnail_url',
    'host_picture_url',
    'host_neighbourhood',
    'host_listings_count',
    'host_total_listings_count',
    'host_verifications',
    'host_has_profile_pic',
    'host_identity_verified',
    'property_type',
    'accommodates',
    'bathrooms',
    'bathrooms_text',
    'bedrooms',
    'beds',
    'amenities',
    'minimum_nights',
    'maximum_nights',
    'minimum_minimum_nights',
    'maximum_minimum_nights',
    'minimum_maximum_nights',
    'maximum_maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'calendar_updated',
    'has_availability',
    'availability_30',
    'availability_60',
    'availability_90',
    'availability_365',
    'calendar_last_scraped',
    'number_of_reviews_ltm',
    'number_of_reviews_l30d',
    'first_review',
    'last_review',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'license',
    'instant_bookable',
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms',
    'reviews_per_month',
    'review_scores_rating',
    'number_of_reviews'
    ]

main_dataframe = main_dataframe.drop(columns=columns_to_drop)

# Renaming columns for consistency
main_dataframe.rename(columns={'neighbourhood_cleansed' : 'neighbourhood'}, inplace=True)
main_dataframe.rename(columns={'room_type' : 'roomtype'}, inplace=True)
main_dataframe.columns

Index(['id', 'name', 'host_id', 'host_name', 'host_since', 'neighbourhood',
       'latitude', 'longitude', 'roomtype', 'price'],
      dtype='object')

### B. Analysing value count

In [22]:
d = {
        'columns': main_dataframe.columns,
        'Non-NA values': main_dataframe.notna().sum(),
        'NA values': main_dataframe.isna().sum(),
        'Total': main_dataframe.isna().sum() + main_dataframe.notna().sum()
    }
value_count_pre_cleaning = pd.DataFrame(data=d)
value_count_pre_cleaning.reset_index(drop=True, inplace=True)
value_count_pre_cleaning

,columns,Non-NA values,NA values,Total
0,id,7707,0,7707
1,name,7707,0,7707
2,host_id,7707,0,7707
3,host_name,7707,0,7707
4,host_since,7707,0,7707
5,neighbourhood,7707,0,7707
6,latitude,7707,0,7707
7,longitude,7707,0,7707
8,roomtype,7707,0,7707
9,price,7522,185,7707


### C. Replacing NaN values

In [23]:
# Removing rows where the rental doesn't have a price
main_dataframe = main_dataframe.dropna(subset=['price'])
main_dataframe['price'].info()

<class 'pandas.core.series.Series'>
Index: 7522 entries, 0 to 7706
Series name: price
Non-Null Count  Dtype 
--------------  ----- 
7522 non-null   object
dtypes: object(1)
memory usage: 117.5+ KB


### D. Removing symbols from numeric columns

In [24]:
# Listing the numeric columns containing symbols such as '%' or '$'
columns_to_clean = [
    'price'
    ]

def remove_non_numeric_characters(df, columns):
    for col in columns:
        df[col] = df[col].str.replace(r'[^0-9\.]', '', regex=True)

remove_non_numeric_characters(main_dataframe, columns_to_clean)

### E. Changing data types

1. Checking dataframe info

In [25]:
main_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7522 entries, 0 to 7706
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7522 non-null   int64  
 1   name           7522 non-null   object 
 2   host_id        7522 non-null   int64  
 3   host_name      7522 non-null   object 
 4   host_since     7522 non-null   object 
 5   neighbourhood  7522 non-null   object 
 6   latitude       7522 non-null   float64
 7   longitude      7522 non-null   float64
 8   roomtype       7522 non-null   object 
 9   price          7522 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 646.4+ KB


2. Changing data types

In [26]:
# Listing columns that must be set to float datatype
columns_to_change_data_type = [
    'price'
    ]

# Function to set float datatype
def change_format(df, columns):
    for col in columns:
        df[col] = df[col].fillna(0).astype(float)

change_format(main_dataframe, columns_to_change_data_type)

# Converting the [host_since] column to the datetime datatype
main_dataframe['host_since'] = main_dataframe['host_since'].astype('datetime64[ns]')

main_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7522 entries, 0 to 7706
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             7522 non-null   int64         
 1   name           7522 non-null   object        
 2   host_id        7522 non-null   int64         
 3   host_name      7522 non-null   object        
 4   host_since     7522 non-null   datetime64[ns]
 5   neighbourhood  7522 non-null   object        
 6   latitude       7522 non-null   float64       
 7   longitude      7522 non-null   float64       
 8   roomtype       7522 non-null   object        
 9   price          7522 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 646.4+ KB


### F. Checking the final version of our main dataframe

1. Dataframe

In [27]:
main_dataframe

,id,name,host_id,host_name,host_since,neighbourhood,latitude,longitude,roomtype,price
0,2384,Hyde Park - Walk to UChicago,2613,Rebecca,2008-08-29,Hyde Park,41.787900,-87.587800,Private room,86.0
1,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,2009-05-19,West Town,41.901660,-87.680210,Entire home/apt,90.0
2,10945,The Biddle House (#1),33004,At Home Inn,2009-08-21,Lincoln Park,41.911960,-87.639810,Entire home/apt,114.0
3,12140,Lincoln Park Guest House,46734,Shay And Robert,2009-10-18,Lincoln Park,41.923570,-87.649470,Private room,329.0
4,24833,Prime LincolnPark 1 Block Fullerton Express L ...,101521,Red,2010-03-31,Lincoln Park,41.925961,-87.656364,Entire home/apt,57.0
...,...,...,...,...,...,...,...,...,...,...
7702,1116851419125146887,Stylish 2BR Retreat in Hyde Park,458971964,Rich,2022-05-13,Hyde Park,41.795290,-87.581940,Entire home/apt,56.0
7703,1117394712148132475,Vintage renovation in Logan Sq!,442832620,Gregory,2022-01-30,Logan Square,41.914243,-87.706237,Entire home/apt,123.0
7704,1117438873812832956,Guest king Room at Aloft Chicago Mag Mile,468436552,Aloft Chicago Mag Mile,2022-07-08,Near North Side,41.892341,-87.622273,Entire home/apt,122.0
7705,1117439997700638787,Guest 2 Queen Room at Aloft Chicago Mag Mile,468436552,Aloft Chicago Mag Mile,2022-07-08,Near North Side,41.894067,-87.621591,Entire home/apt,132.0


2. Counting total values and NA values

In [28]:
d = {
        'columns': main_dataframe.columns,
        'Values': main_dataframe.notna().sum(),
        'NA values': main_dataframe.isna().sum()
    }
value_count_post_cleaning = pd.DataFrame(data=d)
value_count_post_cleaning.reset_index(drop=True, inplace=True)
value_count_post_cleaning

,columns,Values,NA values
0,id,7522,0
1,name,7522,0
2,host_id,7522,0
3,host_name,7522,0
4,host_since,7522,0
5,neighbourhood,7522,0
6,latitude,7522,0
7,longitude,7522,0
8,roomtype,7522,0
9,price,7522,0


# 3. Data modeling

### A. Date dataframe

In [29]:
# Creating a date dataframe with the standard columns used when building and using a data model
# Date range will be based on the oldest year and most recent year found in column [host_since]
min_year = main_dataframe['host_since'].min().date().year
max_year = main_dataframe['host_since'].max().date().year

date = pd.DataFrame({'Date': pd.date_range(f'{min_year}-01-01', f'{max_year}-12-31')})

# Datekey column must have the following structure: 20241231
date['Datekey'] = (date.Date.dt.year.astype(str) + date.Date.dt.month.astype(str).str.zfill(2) + date.Date.dt.day.astype(str).str.zfill(2)).astype(int)
date['Day'] = date.Date.dt.day_name()
date['Day_number'] = date.Date.dt.day
date['Month'] = date.Date.dt.month_name()
date['Month_number'] = date.Date.dt.month
date['Quarter'] = date.Date.dt.quarter
date['Year'] = date.Date.dt.year
date
date.info()

# Setting a variable for the SQL query that will be used to create the table in the database (see section 4.A.)
date_table_create = ("""
    DROP TABLE IF EXISTS date
    CREATE TABLE date(
        date date PRIMARY KEY,
        datekey INTEGER,
        day TEXT,
        day_number INTEGER,
        month TEXT,
        month_number INTEGER,
        quarter INTEGER,
        year INTEGER
    )
""")

,Date,Datekey,Day,Day_number,Month,Month_number,Quarter,Year
0,2008-01-01,20080101,Tuesday,1,January,1,1,2008
1,2008-01-02,20080102,Wednesday,2,January,1,1,2008
2,2008-01-03,20080103,Thursday,3,January,1,1,2008
3,2008-01-04,20080104,Friday,4,January,1,1,2008
4,2008-01-05,20080105,Saturday,5,January,1,1,2008
...,...,...,...,...,...,...,...,...
6205,2024-12-27,20241227,Friday,27,December,12,4,2024
6206,2024-12-28,20241228,Saturday,28,December,12,4,2024
6207,2024-12-29,20241229,Sunday,29,December,12,4,2024
6208,2024-12-30,20241230,Monday,30,December,12,4,2024


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6210 entries, 0 to 6209
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          6210 non-null   datetime64[ns]
 1   Datekey       6210 non-null   int32         
 2   Day           6210 non-null   object        
 3   Day_number    6210 non-null   int32         
 4   Month         6210 non-null   object        
 5   Month_number  6210 non-null   int32         
 6   Quarter       6210 non-null   int32         
 7   Year          6210 non-null   int32         
dtypes: datetime64[ns](1), int32(5), object(2)
memory usage: 267.0+ KB


### B. Hosts dataframe

In [30]:
# The 'Hosts' dataframe is created using main_dataframe by retrieving the unique 'host_id' values and doing a join to retrieve all the 'host' columns from the main_dataframe
hosts_columns = [
    'host_id',
    'host_name',
    'host_since'
    ]

hosts_data_to_retrieve = main_dataframe.loc[:, hosts_columns].dropna().drop_duplicates()

unique_values_hosts = main_dataframe['host_id'].unique().tolist()
hosts_cleaning = pd.DataFrame({'host_id': unique_values_hosts}).dropna()

# Once we have a column with unique host_id, all other hosts data is retrieved by doing a join
hosts = hosts_cleaning.join(hosts_data_to_retrieve.set_index('host_id'), on='host_id', lsuffix='_caller', rsuffix='_other', validate='1:1')

# Renaming columns for consistency
hosts.rename(columns={'host_id' : 'id', 'host_name' : 'name'}, inplace=True)

# Setting a variable for the SQL query that will be used to create the table in the database (see section 4.A.)
hosts_table_create = ("""
    DROP TABLE IF EXISTS hosts
    CREATE TABLE hosts(
        id BIGINT PRIMARY KEY NOT NULL,
        name TEXT NOT NULL,
        host_since DATE NOT NULL
    )
""")

hosts

,id,name,host_since
0,2613,Rebecca,2008-08-29
1,17928,Sarah,2009-05-19
2,33004,At Home Inn,2009-08-21
3,46734,Shay And Robert,2009-10-18
4,101521,Red,2010-03-31
...,...,...,...
3303,168912622,Nour,2018-01-20
3304,567978588,Azhar,2024-03-19
3305,458971964,Rich,2022-05-13
3306,442832620,Gregory,2022-01-30


### C. Neighbourhoods dataframe

In [31]:
unique_values_neighbourhoods = main_dataframe['neighbourhood'].unique().tolist()
neighbourhoods = pd.DataFrame(unique_values_neighbourhoods).dropna()
neighbourhoods['id'] = neighbourhoods.index + 1
neighbourhoods.columns = ['neighbourhood', 'id']

# Changing column order
neighbourhoods = neighbourhoods[['id', 'neighbourhood']]

# Setting a variable for the SQL query that will be used to create the table in the database (see section 4.A.)
neighbourhoods_table_create = ("""
    DROP TABLE IF EXISTS neighbourhoods
    CREATE TABLE neighbourhoods(
        id INTEGER PRIMARY KEY NOT NULL,
        neighbourhood TEXT NOT NULL
    )
""")

neighbourhoods

,id,neighbourhood
0,1,Hyde Park
1,2,West Town
2,3,Lincoln Park
3,4,Logan Square
4,5,Edgewater
...,...,...
72,73,Clearing
73,74,Mount Greenwood
74,75,Riverdale
75,76,Burnside


### D. Roomtypes dataframe

In [32]:
unique_values_roomtypes = main_dataframe['roomtype'].unique().tolist()
roomtypes = pd.DataFrame(data=unique_values_roomtypes).dropna()
roomtypes['id'] = roomtypes.index + 1
roomtypes.columns = ['roomtype', 'id']

# Changing column order
roomtypes = roomtypes[['id', 'roomtype']]

# Setting a variable for the SQL query that will be used to create the table in the database (see section 4.A.)
roomtypes_table_create = ("""
    DROP TABLE IF EXISTS roomtypes
    CREATE TABLE roomtypes(
        id INTEGER PRIMARY KEY NOT NULL,
        roomtype TEXT NOT NULL
        )
""")

roomtypes

,id,roomtype
0,1,Private room
1,2,Entire home/apt
2,3,Shared room
3,4,Hotel room


### E. Rentals dataframe

In [33]:
# Keeping only the [host_id] column from all the hosts-related columns
hosts_columns.remove('host_id')
raw_rentals = main_dataframe.drop(columns=hosts_columns)

# Replacing roomtypes by their corresponding ids
merge1 = pd.merge(raw_rentals, roomtypes, how='left', left_on='roomtype', right_on='roomtype', suffixes=('', '_roomtype'))
merge1.drop(columns=['roomtype'], inplace=True)

# Replacing neighbourhoods by their corresponding ids
merge2 = pd.merge(merge1, neighbourhoods, how='left', left_on='neighbourhood', right_on='neighbourhood', suffixes=('', '_neighbourhood'))
merge2.drop(columns=['neighbourhood'], inplace=True)

# Renaming columns for consistency
merge2.rename(columns={'id_roomtype': 'roomtype_id', 'id_neighbourhood': 'neighbourhood_id'}, inplace=True)

# Setting name 'rentals' for consistency
rentals = merge2

# Setting a variable for the SQL query that will be used to create the table in the database (see section 4.A.)
rentals_table_create = ("""
    DROP TABLE IF EXISTS rentals
    CREATE TABLE rentals(
        id BIGINT PRIMARY KEY NOT NULL,
        name TEXT NOT NULL,
        host_id BIGINT REFERENCES hosts(id) NOT NULL,
        latitude FLOAT NOT NULL,
        longitude FLOAT NOT NULL,
        price NUMERIC NOT NULL,
        roomtype_id INTEGER REFERENCES roomtypes(id) NOT NULL,
        neighbourhood_id INTEGER REFERENCES neighbourhoods(id) NOT NULL
        )
""")

rentals
rentals.info()

,id,name,host_id,latitude,longitude,price,roomtype_id,neighbourhood_id
0,2384,Hyde Park - Walk to UChicago,2613,41.787900,-87.587800,86.0,1,1
1,7126,Tiny Studio Apartment 94 Walk Score,17928,41.901660,-87.680210,90.0,2,2
2,10945,The Biddle House (#1),33004,41.911960,-87.639810,114.0,2,3
3,12140,Lincoln Park Guest House,46734,41.923570,-87.649470,329.0,1,3
4,24833,Prime LincolnPark 1 Block Fullerton Express L ...,101521,41.925961,-87.656364,57.0,2,3
...,...,...,...,...,...,...,...,...
7517,1116851419125146887,Stylish 2BR Retreat in Hyde Park,458971964,41.795290,-87.581940,56.0,2,1
7518,1117394712148132475,Vintage renovation in Logan Sq!,442832620,41.914243,-87.706237,123.0,2,4
7519,1117438873812832956,Guest king Room at Aloft Chicago Mag Mile,468436552,41.892341,-87.622273,122.0,2,16
7520,1117439997700638787,Guest 2 Queen Room at Aloft Chicago Mag Mile,468436552,41.894067,-87.621591,132.0,2,16


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7522 entries, 0 to 7521
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                7522 non-null   int64  
 1   name              7522 non-null   object 
 2   host_id           7522 non-null   int64  
 3   latitude          7522 non-null   float64
 4   longitude         7522 non-null   float64
 5   price             7522 non-null   float64
 6   roomtype_id       7522 non-null   int64  
 7   neighbourhood_id  7522 non-null   int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 470.3+ KB


# 4. Importing dataframes

### A. Creating tables in database

In [34]:
# Initiating a pyodbc connection to the database
db = pyodbc.connect(f'DRIVER=SQL SERVER;SERVER={SERVER};DATABASE={DATABASE}')
cursor = db.cursor()

# Listing all the tables to create in the database through SQL queries
# These tables are empty and will be populated with values in the next section by importing the corresponding dataframe created in section 3.
table_creation = [
    date_table_create, 
    hosts_table_create, 
    neighbourhoods_table_create, 
    roomtypes_table_create, 
    rentals_table_create
]

# Creating tables in database
for table in table_creation:
        cursor.execute(table)
        db.commit()

# Closing pyodbc connection to the database
cursor.close()
db.close()

### B. Importing dataframes to their corresponding tables

In [35]:
# Once the tables have been created in the previous section (4.A.), the dataframes are imported to their corresponding table in the database
# The dataframes were created throughout section 3
date.to_sql('date', con=connection, if_exists='append', index=False)
hosts.to_sql('hosts', con=connection, if_exists='append', index=False)
neighbourhoods.to_sql('neighbourhoods', con=connection, if_exists='append', index=False)
roomtypes.to_sql('roomtypes', con=connection, if_exists='append', index=False)
rentals.to_sql('rentals', con=connection, if_exists='append', index=False)

184

512

77

4

186

### C. Checking database tables through queries

In [36]:
# Listing the name of the tables created
tables = [
    'date',
    'hosts',
    'neighbourhoods',
    'roomtypes',
    'rentals'
]

# SQL query : "select *" from each table
for table in tables:
    pd.read_sql(sql=f'select * from {table}', con=connection)

,date,datekey,day,day_number,month,month_number,quarter,year
0,2008-01-01,20080101,Tuesday,1,January,1,1,2008
1,2008-01-02,20080102,Wednesday,2,January,1,1,2008
2,2008-01-03,20080103,Thursday,3,January,1,1,2008
3,2008-01-04,20080104,Friday,4,January,1,1,2008
4,2008-01-05,20080105,Saturday,5,January,1,1,2008
...,...,...,...,...,...,...,...,...
6205,2024-12-27,20241227,Friday,27,December,12,4,2024
6206,2024-12-28,20241228,Saturday,28,December,12,4,2024
6207,2024-12-29,20241229,Sunday,29,December,12,4,2024
6208,2024-12-30,20241230,Monday,30,December,12,4,2024


,id,name,host_since
0,2153,Linda,2008-08-16
1,2613,Rebecca,2008-08-29
2,4434,Kellen,2008-11-20
3,6162,Jackie,2009-01-08
4,7529,Emily,2009-02-07
...,...,...,...
3303,565344683,Flora,2024-03-04
3304,565714549,Gigi,2024-03-06
3305,566397122,Alis,2024-03-10
3306,567068804,City Suites Hotel,2024-03-14


,id,neighbourhood
0,1,Hyde Park
1,2,West Town
2,3,Lincoln Park
3,4,Logan Square
4,5,Edgewater
...,...,...
72,73,Clearing
73,74,Mount Greenwood
74,75,Riverdale
75,76,Burnside


,id,roomtype
0,1,Private room
1,2,Entire home/apt
2,3,Shared room
3,4,Hotel room


,id,name,host_id,latitude,longitude,price,roomtype_id,neighbourhood_id
0,2384,Hyde Park - Walk to UChicago,2613,41.787900,-87.587800,86.0,1,1
1,7126,Tiny Studio Apartment 94 Walk Score,17928,41.901660,-87.680210,90.0,2,2
2,10945,The Biddle House (#1),33004,41.911960,-87.639810,114.0,2,3
3,12140,Lincoln Park Guest House,46734,41.923570,-87.649470,329.0,1,3
4,24833,Prime LincolnPark 1 Block Fullerton Express L ...,101521,41.925961,-87.656364,57.0,2,3
...,...,...,...,...,...,...,...,...
7517,1116851419125146887,Stylish 2BR Retreat in Hyde Park,458971964,41.795290,-87.581940,56.0,2,1
7518,1117394712148132475,Vintage renovation in Logan Sq!,442832620,41.914243,-87.706237,123.0,2,4
7519,1117438873812832956,Guest king Room at Aloft Chicago Mag Mile,468436552,41.892341,-87.622273,122.0,2,16
7520,1117439997700638787,Guest 2 Queen Room at Aloft Chicago Mag Mile,468436552,41.894067,-87.621591,132.0,2,16


### D. Closing connection to database

In [37]:
# Closing the sqlalchemy engine connection to the database
connection.close()